This folder collects the data given the csv. 

Load requisite libraries

In [25]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd 
#from sklearn.cluster import KMeans
#import matplotlib.pyplot as plt
import numpy as np
from requests.exceptions import RequestException
import time


Build methods for cpaturing relevant html content

In [2]:
#hello TN

#methods
#total number of words in page     
def total_Words(soup):
    total_words = 0
    paragraphs = soup.find_all("p")
    for p in paragraphs: 
        total_words += len(p.get_text().split())
    return total_words
    
    
def total_Characters(soup): 
    total_char = 0
    paragraph = soup.find_all("p")
    for p in paragraph: 
        total_char += len(p.get_text())
    return total_char
    
    
#total number of references 
def total_Ref(soup): 
    references_section = soup.find('ol', class_= 'references')

    if references_section:
        references = references_section.find_all('li')
        num_references = len(references)
       # print(f"Number of references: {num_references}")
        return num_references
    else:
        #print("No references found.")    
        return 0
    
#end of methods

Load data and create a subset for testing

In [3]:
birdSpecies = pd.read_csv("../data/speciesList.csv")

testSubset = birdSpecies.iloc[0:8375, 1]
#testSubset = birdSpecies.iloc[0:25, 1]
print(testSubset)

0       Accipiter albogularis
1            Accipiter badius
2           Accipiter bicolor
3        Accipiter brachyurus
4          Accipiter brevipes
                ...          
8369        Trogon personatus
8370             Trogon rufus
8371         Trogon surrucura
8372         Trogon violaceus
8373           Trogon viridis
Name: Species1, Length: 8374, dtype: object


Build the scrape. Start by creating lists of the right length, then save the base url as a variable. After that, use a for loop to first check the status code of a given species; if it's 200, proceed as usual, if it's not, upate 'Error' to hold the non 200 status code. This is actually a nice example, as it shows that if a species does not return a page, it actually produces a 404 error, so that's one less check that you need to do.

In [4]:
# create empty lists of the appropriate size for each variable being captured
NumReferences = [None] * len(testSubset)
NumChar = [None] * len(testSubset)
NumWords = [None] * len(testSubset)
Error = [None] * len(testSubset)

wikipedia_base = "https://en.wikipedia.org/wiki/"


df = pd.DataFrame(columns=["SpeciesName","NumReferences", "NumChar", "NumWords", "Error"])

start = time.time()


for i in range(len(testSubset)):
    
    url = wikipedia_base + testSubset.iloc[i]
    response = rq.get(url)
    
    html_doc = rq.get(url).text
    errorBoolean = True
        
    soup = bs(html_doc, 'html.parser')
        
    
    if response.status_code == 200:
        Error[i] = response.status_code
        html_doc = response.text
        # in here you now handle the html code
    else:
        Error[i] = response.status_code
    
    
    print("printing species: " + str(i))
    time.sleep(3)
    print(testSubset.iloc[i])
    print(Error[i])
    print("done")
    a = total_Characters(soup)
    b = total_Words(soup)
    c = total_Ref(soup)
  #  print(a)
  #  print(b)
  #  print(c)
    
    
    new_row = pd.DataFrame({"SpeciesName": [testSubset.iloc[i]], "NumReferences": [c], "NumChar": [a], "NumWords": [b], "Error": 
                               [Error[i]]})
    df = pd.concat([df, new_row], ignore_index=True)
  
end = time.time()
    
total = end - start 
    
print("total execution time is: " + str(total))
        
#print(Error)
#print(testSubset)
print(df)


printing species: 0
Accipiter albogularis
200
done
printing species: 1
Accipiter badius
200
done
printing species: 2
Accipiter bicolor
200
done
printing species: 3
Accipiter brachyurus
200
done
printing species: 4
Accipiter brevipes
200
done
printing species: 5
Accipiter butleri
200
done
printing species: 6
Accipiter castanilius
200
done
printing species: 7
Accipiter cirrocephalus
200
done
printing species: 8
Accipiter collaris
200
done
printing species: 9
Accipiter cooperii
200
done
printing species: 10
Accipiter erythrauchen
200
done
printing species: 11
Accipiter erythropus
200
done
printing species: 12
Accipiter fasciatus
200
done
printing species: 13
Accipiter francesiae
200
done
printing species: 14
Accipiter gentilis
200
done
printing species: 15
Accipiter griseiceps
200
done
printing species: 16
Accipiter gularis
200
done
printing species: 17
Accipiter gundlachi
200
done
printing species: 18
Accipiter haplochrous
200
done
printing species: 19
Accipiter henicogrammus
200
done
pr

In [29]:
# getting rid of the species that have errors 

subsetDF_200 = df[df["Error"] == 200]
subsetDF_other = df[df["Error"] != 200]

print(subsetDF_200)
print("hello")

print(subsetDF_other)



                  SpeciesName NumReferences NumChar NumWords Error
0       Accipiter albogularis             2     359       56   200
1            Accipiter badius            35    6188     1023   200
2           Accipiter bicolor            17    5405      863   200
3        Accipiter brachyurus             3    1646      278   200
4          Accipiter brevipes            14    4300      716   200
..                        ...           ...     ...      ...   ...
99      Buteogallus coronatus            13    5868      963   200
100    Buteogallus gundlachii             4    2150      359   200
101  Buteogallus lacernulatus             4    2867      470   200
102  Buteogallus meridionalis             7    1399      236   200
103   Buteogallus schistaceus             2     846      135   200

[104 rows x 5 columns]
hello
Empty DataFrame
Columns: [SpeciesName, NumReferences, NumChar, NumWords, Error]
Index: []


In [28]:
subsetDF_200.to_csv("../data/subsetSpecies_200.csv")
subsetDF_other.to_csv("../data/subsetSpecies_other.csv")


In [88]:
print(soup.title.string) 
print(soup.find("h2", {"id": "References"})) #if it doesn't find anything, it will return none. If it does find it, it will return tag (h2) and the text 
#h2Ref = (soup.find_all("h2"))
#print(h2Ref)
#type(h2Ref)
#h2Ref.tag[id]

#looks to see if there is an h2 with the id references. If there is, do [X], and if not, do [Y]. 
#swap out whatever criteria Sandra deems sufficient evidence 
#refences is more guaranteed to exist as a header
#extract number of words in the wikipedia page, ... is that sufficient evidence??
#list of all species 
#construct a list of boolean values 
#if eveidcene exist, 
# Bird1 - True
# Bird2- False 
#Bird3 - True 

#use pandas to create a data frame 
#combining those 2 lists to create a rectangular dataset and create a csv 
# 2 variables: name, boolean value (T or F)

Bare-throated bellbird - Wikipedia
<h2 id="References">References</h2>


In [4]:
#everyhting we have dones so far assume wikipedia entries ... what if it does not exist
#find a weak speceis and look to see what returns 
#unique identify of no article exists 
# then put a false 
#What to do if species dne on wikipedia 

#assuming we successfully connected to wikipedia and there is no server error 

# number 1 thing to do is verfiy there is no server error!! 
# request library that talks to the server allows to figure out what type of error 
# server number is 200: which means it works 
#anything else is invalid 
# a list of server response codes 
#request if response code is 200 
#look into species 
#if response code is not 200, record a false in the boolean list 

#do a few for a test 
#what are limits to scraping wikipedia ... look up on google 
#what is the number per second, per day, per week...


In [11]:
import pandas as pd

df = pd.DataFrame(columns=["SpeciesName", "NumReferences", "NumParagraphs", "NumWords", "Error"])



birdSpecies = ["Accipiter albogularis", "Accipiter badius", "Accipiter bicolor"]

    
for i in birdSpecies:
    # Create a DataFrame for the new row
    new_row = pd.DataFrame({"SpeciesName": [i], "NumReferences": [num_references], "NumParagraphs": [totalParagraph], "NumWords": [totalWords]})
    
    # Append the new row to the existing DataFrame
    df = pd.concat([df, new_row], ignore_index=True)

# Display the DataFrame
print(df)

['Accipiter albogularis', 'Accipiter badius', 'Accipiter bicolor']
Accipiter albogularis
Accipiter badius
Accipiter bicolor
             SpeciesName NumReferences NumParagraphs NumWords Error
0  Accipiter albogularis            35            12     6188   NaN
1       Accipiter badius            35            12     6188   NaN
2      Accipiter bicolor            35            12     6188   NaN
